# Database tool to store portfolio data for DCA strategy

Install and create the SQLite DB

In [1]:
!pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
%sql sqlite://

In [4]:
import pandas as pd
import sqlite3

Create a table and DB (if not created yet)

In [66]:
def create_table():
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    try:
        # actual value (from last input POV)
        c.execute("""
            CREATE TABLE DCA_portfolio (
            company_name text,
            ticker text,
            shares integer,
            last_buy_price real,
            avg_price real,
            invested_value real,
            actual_value real,
            profit real,
            share_in_portfolio real
            )""")
        conn.commit()
        conn.close()
    except Exception as e:
        conn.commit()
        conn.close()
        print(e)

In [67]:
create_table()

table DCA_portfolio already exists


Insert some test data statically

In [68]:
def insert_test_data():
    ''' Used just for initial testing '''
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("INSERT INTO DCA_portfolio VALUES (?,?,?,?,?,?,?,?,?)", ('company_name', 'xxx', 100, 50.2, 48.25, 4825, 5020, 195, 1))
    c.execute("INSERT INTO DCA_portfolio VALUES (?,?,?,?,?,?,?,?,?)", ('company_name2', 'yyy', 75, 102.5, 102.5, 7687.5, 7687.5, 0, 1))
    conn.commit()
    conn.close()

In [69]:
# insert_test_data()

Show the portfolio with test data

In [70]:
def show_portfolio():
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("SELECT * FROM DCA_portfolio")
    table = c.fetchall()
    conn.commit()
    conn.close()
    return table

In [10]:
show_portfolio()

[('company_name', 'xxx', 100, 50.2, 48.25, 4825.0, 5020.0, 195.0, 1.0),
 ('company_name2', 'yyy', 75, 102.5, 102.5, 7687.5, 7687.5, 0.0, 1.0)]

Create the functions needed to calculate values for new portfolio position

In [21]:
def get_last_buy_price(c, ticker, price):
    c.execute("SELECT last_buy_price FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    lbp = c.fetchall()
    return lbp[0][0] if lbp else price

In [22]:
def get_avg_price(c, ticker, price, shares):
    c.execute("SELECT avg_price FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    ap = c.fetchall()
    if not ap:
        return price
    else:
        c.execute("SELECT shares FROM DCA_portfolio WHERE ticker = ?", (ticker,))
        ash = c.fetchall()[0][0]
        ap = ap[0][0]
        ap = (ap * ash + shares * price)/(ash+shares)
        return ap

In [23]:
# get invested value
def get_invested_value(c, ticker, price, shares):
    c.execute("SELECT invested_value FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    iv = c.fetchall()
    if not iv:
        return price*shares
    else:
        return iv[0][0]+(price*shares)

In [24]:
# get actual value (from last input POV)
def get_actual_value(c, ticker, price, shares):
    c.execute("SELECT actual_value FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    av = c.fetchall()
    if not av:
        return price*shares
    else:
        c.execute("SELECT shares FROM DCA_portfolio WHERE ticker = ?", (ticker,))
        ash = c.fetchall()[0][0]
        return price*(shares+ash)

In [52]:
def get_share_in_portfolio(c, actual_value):
    ''' get the current size of the position in the portfolio [%] '''
    c.execute("SELECT actual_value FROM DCA_portfolio ")
    psvl = c.fetchall()
    if not psvl:
        return 100
    else:
        avl = [x[0] for x in psvl]
        return 100*actual_value/(sum(avl)+actual_value) # sum of existing + the new position

Function for adding new position into portfolio

In [35]:
# would be better to perform it as a class
def add_asset(company_name, ticker, shares, price):
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    
    last_buy_price = get_last_buy_price(c, ticker, price)
    avg_price = get_avg_price(c, ticker, price, shares)
    invested_value = get_invested_value(c, ticker, price, shares)
    actual_value = get_actual_value(c, ticker, price, shares)
    profit = 100*(actual_value-invested_value)/invested_value
    share_in_portfolio = get_share_in_portfolio(c, actual_value) # that will need to be updated (the values to be wrong after adding new asset)
    
    c.execute("INSERT INTO DCA_portfolio VALUES (?,?,?,?,?,?,?,?,?)", (company_name, ticker, shares, last_buy_price, avg_price, invested_value, actual_value, profit, share_in_portfolio))

    conn.commit()
    conn.close()

Let's add a test asset

In [36]:
add_asset('test', 't2', 200, 37.5)

After adding at least one asset the share_in_portfolio column needs to be updated

In [71]:
def update_the_share_in_portfolio():
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("SELECT company_name, actual_value FROM DCA_portfolio ") # instead of the company name the ticker can be used as well
    psvl = c.fetchall()
    avs = sum([x[1] for x in psvl])
    for comp, value in psvl:
        c.execute("UPDATE DCA_portfolio SET share_in_portfolio = ? WHERE company_name = ?", (100*value/(avs), comp))
    conn.commit()
    conn.close()

In [ ]:
update_the_share_in_portfolio()

Let's verify the updates

In [63]:
show_portfolio()

[('company_name',
  'xxx',
  100,
  50.2,
  48.25,
  4825.0,
  5020.0,
  195.0,
  24.84226153655821),
 ('company_name2',
  'yyy',
  75,
  102.5,
  102.5,
  7687.5,
  7687.5,
  0.0,
  38.042805888902635),
 ('test', 't2', 200, 37.5, 37.5, 7500.0, 7500.0, 0.0, 37.114932574539154)]

In [ ]:
def update_asset(company_name, ticker, shares, price):
    # to do similar operations for existing companies (verify the comp name and ticer)